In [1]:
import re
import requests

# Function to check for suspicious patterns in a URL
def check_url_pattern(url):
    # Patterns that are commonly found in phishing URLs
    phishing_patterns = [
        r"http[s]?://.*@.*",  # Embedded credentials
        r"http[s]?://.*[0-9]+\.[0-9]+\.[0-9]+\.[0-9]+.*",  # IP addresses
        r".*\?.*=.*=.*",  # Multiple query parameters
        r".*\%[0-9a-fA-F]{2}.*",  # URL encoding
    ]
    for pattern in phishing_patterns:
        if re.search(pattern, url):
            return True
    return False

# Function to check URL structure
def check_url_structure(url):
    suspicious_chars = ["-", "_", ".", "=", "&"]
    if len(url.split("/")) > 6:  # Too many segments in URL
        return True
    if any(char * 3 in url for char in suspicious_chars):  # Repeated special characters
        return True
    return False

# Function to query a blacklist (a simple static list for demonstration)
def check_blacklist(url, blacklist):
    domain = re.findall(r"https?://([A-Za-z_0-9.-]+).*", url)
    domain = domain[0] if domain else url
    return domain in blacklist

# Function to query VirusTotal (requires an API key)
def check_virustotal(url, api_key):
    headers = {"x-apikey": api_key}
    response = requests.get(f"https://www.virustotal.com/api/v3/urls/{url}", headers=headers)
    if response.status_code == 200:
        result = response.json()
        malicious_count = result.get("data", {}).get("attributes", {}).get("last_analysis_stats", {}).get("malicious", 0)
        return malicious_count > 0
    return False

# Main function
def phishing_link_scanner(url, blacklist, api_key=None):
    if check_url_pattern(url):
        print(f"ALERT: Suspicious URL pattern detected: {url}")
    elif check_url_structure(url):
        print(f"ALERT: Unusual URL structure detected: {url}")
    elif check_blacklist(url, blacklist):
        print(f"ALERT: URL found in blacklist: {url}")
    elif api_key and check_virustotal(url, api_key):
        print(f"ALERT: URL flagged as malicious by VirusTotal: {url}")
    else:
        print(f"URL seems safe: {url}")

# Example Usage
if __name__ == "__main__":
    # Example blacklist
    blacklist = [
        "malicious.com",
        "phishing-site.org",
        "badlink.net",
    ]

    # Input URL to test
    test_url = input("Enter URL to scan: ")

    # Replace 'your_api_key' with a real VirusTotal API key
    api_key = "your_api_key"

    phishing_link_scanner(test_url, blacklist, api_key)


Enter URL to scan:  https://classroom.google.com/


URL seems safe: https://classroom.google.com/
